<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [2]:
!pip install optuna
!pip install LAMDA-SSL

Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection

# Modell
from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

In [5]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [7]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import umarkert_andel_dag
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts_semi

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_semiveiledet import BHD
from moduler_optimalisering_evaluering_semiveiledet import SelfTrainingRandomForestRegressor

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_dag

from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_1
from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_2

from moduler_optimalisering_evaluering_semiveiledet import bhd_andel_maks
from moduler_optimalisering_evaluering_semiveiledet import andel_umarkert

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av res

In [12]:
# Oppretter instans for lagring av res
res_semi = Resultat_dicts_semi()

# Setter fast testandel på 20%, definerer treningsandeler og markerte andeler

In [13]:
# Fast testsandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0

# Markerte andeler til undersøkelse
andel_av_markert = [0.5, 1.0, 2.0]

# Parametervalg for hyperoptimalisering

In [20]:
# Antall runder for hyperoptimalisering
n_runder_hel = 75
n_runder_andel = 75

# Antall gjentagelser
n_gjentagelser = 1

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [16]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av satt markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andel
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt,
                          dict_res=res_semi.dict_res_1_hel,
                          rå_data=rå_data,
                          treningssett_markert=trening_markert,
                          enzymtyper_treningssett_markert=enzymtyper_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          n_splitt=n_splitt,
                          n_gjentagelser=n_gjentagelser,
                          n_runder=n_runder_hel,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

# Lagring av resultater
res_1_hel_semi = pd.DataFrame(res_semi.dict_res_1_hel)
res_1_hel_opt_semi = pd.DataFrame(res_semi.dict_res_1_hel_opt)

res_1_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_hel_semi.csv')
res_1_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_hel_opt_semi.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt,
                            dict_res=res_semi.dict_res_2_hel,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

# Lagring av resultater
res_2_hel_semi = pd.DataFrame(res_semi.dict_res_2_hel)
res_2_hel_opt_semi = pd.DataFrame(res_semi.dict_res_2_hel_opt)

res_2_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_hel_semi.csv')
res_2_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_hel_opt_semi.csv')

Beste modell

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)


  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt,
                            dict_res=res_semi.dict_res_1_design,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            enzymtyper_treningssett_markert=enzymtyper_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_gjentagelser=n_gjentagelser,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

# Lagring av resultater
res_1_design_semi = pd.DataFrame(res_semi.dict_res_1_design)
res_1_design_opt_semi = pd.DataFrame(res_semi.dict_res_1_design_opt)

res_1_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_design_semi.csv')
res_1_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_design_opt_semi.csv')


[I 2024-04-22 20:26:46,972] A new study created in memory with name: no-name-d4765988-5d3a-480e-bcdc-0d26f5cf70fa
[I 2024-04-22 20:39:08,606] Trial 0 finished with value: 773.5818846632824 and parameters: {'k1': 18, 'k2': 14, 'p1': 7, 'p2': 7, 'max_iters': 84, 'pool_size': 123}. Best is trial 0 with value: 773.5818846632824.
[I 2024-04-22 20:45:44,168] A new study created in memory with name: no-name-7789f484-46ad-434a-91d5-43b0ddf9a098
[I 2024-04-22 20:46:29,680] Trial 0 finished with value: 744.7789457467345 and parameters: {'n_estimators': 665, 'criterion': 'friedman_mse', 'max_depth': 28, 'min_samples_leaf': 2, 'max_features': 0.6373427447180475, 'maks_iterasjoner': 10, 'std_terskel': 0.029886293317025428}. Best is trial 0 with value: 744.7789457467345.
[I 2024-04-22 20:47:44,856] A new study created in memory with name: no-name-4b2e5912-0342-41d8-8d42-379377e7806c
[I 2024-04-22 20:48:16,205] Trial 0 finished with value: 1627.7980521794639 and parameters: {'k': 1069, 'graf_vektet_h

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  #Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt,
                        dict_res=res_semi.dict_res_2_design,
                        rå_data=rå_data,
                        treningssett_markert=trening_markert,
                        treningssett_umarkert=treningssett_umarkert,
                        treningssett=treningssett,
                        testsett=testsett,
                        n_splitt=n_splitt,
                        n_runder=n_runder_andel,
                        test_andel=test_andel,
                        trenings_andel=trenings_andel,
                        random_seed=random_seed)

# Lagring av resultater
res_2_design_semi = pd.DataFrame(res_semi.dict_res_2_design)
res_2_design_opt_semi = pd.DataFrame(res_semi.dict_res_2_design_opt)

res_2_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_design_semi.csv')
res_2_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_design_opt_semi.csv')


# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt_NIR,
                         dict_res=res_semi.dict_res_1_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
res_1_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_NIR)
res_1_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_opt_NIR)

res_1_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_hel_semi_NIR.csv')
res_1_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_hel_opt_semi_NIR.csv')

## Alternativ 2: Manuell fordeling

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt_NIR,
                         dict_res=res_semi.dict_res_2_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
res_2_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_NIR)
res_2_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_opt_NIR)

res_2_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_hel_semi_NIR.csv')
res_2_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_hel_opt_semi_NIR.csv')

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt_NIR,
                         dict_res=res_semi.dict_res_1_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
res_1_design_trening_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_NIR)
res_1_design_test_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_opt_NIR)

res_1_design_trening_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_design_semi_NIR.csv')
res_1_design_test_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_1_design_opt_semi_NIR.csv')

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt_NIR,
                         dict_res=res_semi.dict_res_2_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
res_2_design_trening_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_NIR)
res_2_design_test_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_opt_NIR)

res_2_design_trening_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_design_semi_NIR.csv')
res_2_design_test_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_res_2_design_opt_semi_NIR.csv')